In [ ]:
import os, sys
dir1 = os.path.abspath('..\..')
sys.path.append(dir1+ "\\")

import MetaTrader5 as mt
from datetime import datetime as dt

# custom
from my_jupyter.user import User
from my_jupyter.strategy_manager import StrategyManager
from my_jupyter.operation import Operation
from my_jupyter.strategies.min_max_strategy import MinMaxStrategy
from my_jupyter.market_data_repository import MarketDataRepository
# from Mbox import Mbox
# from boleta_1 import gerar_boleta
# from gerenciamento import Gerenciamento

## GUI's
from daemon_tools import DaemonThreadMini


In [ ]:
strategies = [MinMaxStrategy()]
operation = Operation("WINV23", 1, mt.TIMEFRAME_M1, True, True)
user = User(strategies=strategies,operations=[operation], trading_time_start=dt.now())
market_data_repository = MarketDataRepository()
strategy = StrategyManager(market_data_repository, user)

# PARAMETROS MAIN
kwargs = {
    'strategy': strategy,
}

def main(**kwargs):
    _shift= 0 
    strategy: StrategyManager = kwargs['strategy']
    strategy.run_strategies()
    # print(f"{dt.now().strftime('%d/%m/%Y, %H:%M:%S')} >>> D-{_shift}")

In [ ]:
def exec_daemon(**kwargs):
    main(**kwargs)
    # alarme.ring_alarme()   

In [ ]:
# gerenciamento = Gerenciamento(mt, timeframe, timeframes_dict)

try:
    main(**kwargs)
except Exception as e:
    print(e.with_traceback(__tb=None))
    # Mbox.Alerta("ERRO", "busca de sinais com erro")


In [ ]:
from my_jupyter.daemons.daemon_tools import DaemonThreadMini


thread_exec = DaemonThreadMini(exec_daemon, seconds=0, mseconds=100, **kwargs)
thread_exec.start()

In [ ]:
from datetime import timedelta as td, datetime as dt
import pandas as pd
import numpy as np
import threading
import time
from matplotlib import pyplot as plt
from my_jupyter.metatrader_wrapper import MetatraderWrapper
from IPython.display import display, clear_output

mt = MetatraderWrapper()
mt.demo_on()

td_ = td(hours=3, minutes=5)
now = dt.now()
nexti = dt(now.year, now.month, now.day, now.hour, now.minute, now.second) - td_
# print(nexti)
# ticks = market_data_repository.mt.copy_ticks_from("WINV23", nexti,100, mt.TICK_FLAG_VOLUME)
# fig = plt.figure()
# ax = fig.add_subplot(111)
# line1, = ax.plot([], [], 'r-') # Returns a tuple of line objects, thus the comma
# Enable interactive mode for Matplotlib
plt.ion()

# Create initial empty plot
fig, ax = plt.subplots()
(line,) = ax.plot([], [])
(avg_,) = ax.plot([], [])
ax.set_ylim(0, 10)
ax.set_xlim(0, 2*60+1)
fig.set_figheight(6)
fig.set_figwidth(8)

y_max = -1
# plt.show()
while True:
    now = dt.now()
    nexti = now - td_
    ticks = mt.mt.copy_ticks_range("WINV23", nexti, now, mt.mt.COPY_TICKS_ALL)

    ticks_frame = pd.DataFrame(ticks)

    # ticks_frame["time"] = pd.to_datetime(ticks_frame["time_msc"]/1000, unit="s")
    ticks_frame["time"] = round(ticks_frame["time_msc"] / 1000)
    ticks_frame["time_dt"] = pd.to_datetime(round(ticks_frame["time_msc"]/1000), unit="s")

    # ticks_frame.groupby(ticks_frame['time'].dt.second)['volume'].agg('sum')
    # data = ticks_frame.groupby([ticks_frame['time'].dt.minute, ticks_frame['time'].dt.second]).agg(volume=('volume','sum'))
    data = ticks_frame.groupby([ticks_frame["time_dt"]]).agg(
        volume=("volume", "sum")
    )

    a = pd.Series(list(data.values)).rolling(10).mean()
    y = list(map(lambda x: x, data.values))
    # x = list(map(lambda x: dt(x), data.index -min(x)))
    x = list(map(lambda x: x, data.index ))
    y_max = max(y)
    ax.set_ylim(0,y_max)
    ax.set_xlim(data.index[0], data.index[-1] )
    print(nexti, now)

    # ax.set_xlim(0,len(x)+1)

    print(len(x), x)
    print(len(y), y)
    line.set_data(x, y)
    avg_.set_data(x,a)

    # Redraw the plot
    display(fig)
    clear_output(wait=True)
    time.sleep(2)

    # print("Hiiiiiiiiiiii")
    # ax.clear()
    # # ax.plot(data.index, data.values)
    # print("show")
    # fig.canvas.draw()
    # fig.canvas.flush_events()
    # plt.show()

    # refresh()
    # t =threading.Thread(target=refresh)
    # t.start()
    # print(data.index, data.values)

In [ ]:
import threading
import time
def print_it():
    while True:
        print("hi")
        time.sleep(1)
t = threading.Thread(target=lambda : print_it())
t.daemon = True
t.start()

In [ ]:
bars = mt.mt.copy_rates_from_pos("PETR4", mt.mt.TIMEFRAME_D1, 0, 1000000-1)[::-1]


In [ ]:
bars = mt.mt.copy_rates_from_pos("WIN$", mt.mt.TIMEFRAME_D1, 0, 1000000 - 1)[::-1]


In [ ]:
bars = mt.mt.copy_rates_from_pos("PETR4", mt.mt.TIMEFRAME_D1, 0, 1000000 - 1)[::-1]


In [ ]:
bars = mt.mt.copy_rates_from_pos("WIN$", mt.mt.TIMEFRAME_M1, 0, 100000-1)[::-1]


In [ ]:
bars = mt.mt.copy_rates_from_pos("WIN$", mt.mt.TIMEFRAME_M5, 0, 100000 - 1)[::-1]
